In [1]:
from csoundengine import *
import logging
# logging.basicConfig(level="DEBUG")


In [2]:
e = Engine()
s = e.session()

--Csound version 7.0 (double samples) Feb 25 2025
[commit: 2d7b783a0ed4e67b6764d69cfb29edefdaea8826]
libsndfile-1.2.2
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
audio buffered in 512 sample-frame blocks
reading 4096-byte blks of shorts from adc:Built-in Audio Analog Stereo: (RAW)
writing 1024 sample blks of 64-bit floats to dac:Built-in Audio Analog Stereo:
SECTION 1:


In [3]:
foo = s.defInstr('foo', r'''
; foo instr
; Args:
;   kfreq: frequency
if changed2(kfreq) == 1 then
    println "kfreq: %f", kfreq
endif
outch 1, oscili:a(0.1, kfreq) * linsegr:a(0, 0.01, 1, 0.01, 0)
''', args={'kfreq': 800})
foo

Instr(foo, kfreq=800)

In [4]:
print(foo.generateBody())

; foo instr
; Args:
;   kfreq: frequency
kfreq = p5
if changed2(kfreq) == 1 then
    println "kfreq: %f", kfreq
endif
outch 1, oscili:a(0.1, kfreq) * linsegr:a(0, 0.01, 1, 0.01, 0)
atstop dict_get:i(gi__builtinInstrs, "notifyDealloc"), 0, 0, p1


In [5]:
sy = s.sched('foo', 0, 10, kfreq=800)
# sy.engine.callLater(0.5, sy.stop)
sy

Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 foo=501.0002 start=2.483 dur=10.000 p5=800)

In [6]:
sy.stop()


TypeError: Session._deallocSynthResources() takes 2 positional arguments but 3 were given

In [8]:
sy.set(kfreq=502)

kfreq: 502.000000


In [9]:
foo = s.defInstr('foo', r'''
{kfreq=1000}
if changed2(kfreq) == 1 then
    println "kfreq: %f", kfreq
endif
outch 1, oscili:a(0.1, kfreq) * linsegr:a(0, 0.01, 1, 0.01, 0)
''')
foo

Instr(foo, kfreq=1000)

In [10]:
foo.dynamicParamNames()

frozenset({'kfreq'})

In [11]:
sy = s.sched('foo', kfreq=700)
sy

Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 foo=501.0004 start=20.105 dur=-1.000 p5=700)

In [12]:
sy.args

[700.0]

In [13]:
sy.set(kfreq=3140)

kfreq: 3140.000000


In [14]:
print(sy.body)

kfreq = p5
if changed2(kfreq) == 1 then
    println "kfreq: %f", kfreq
endif
outch 1, oscili:a(0.1, kfreq) * linsegr:a(0, 0.01, 1, 0.01, 0)
atstop 1, 0.01, 0.01, p1


In [15]:
bar = s.defInstr('bar', r'''
|kfreq=1000|
if changed2(kfreq) == 1 then
    println "kfreq: %f", kfreq
endif
outch 1, oscili:a(0.1, kfreq) * linsegr:a(0, 0.01, 1, 0.01, 0)
''')
bar

Instr(bar, kfreq=1000)

In [16]:
baz = s.defInstr('baz', r'''
pset p1, p2, p3, 0, 440, 0.1
kfreq = p5
iamp = p6
outch 1, oscili:a(iamp, kfreq) * linsegr:a(0, 0.01, 1, 0.01, 0)
''')
baz

Instr(baz, kfreq=440, iamp=0.1)

In [17]:
def callback(*args):
    # import time
    print("**************", args)
    # time.sleep(0.01)

sy = s.sched('baz', dur=2, kfreq=800, iamp=0.1, whenfinished=lambda *args: print("*********", args))
sy

Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 baz=502.0002 start=35.742 dur=2.000 p5=800 kfreq=0.1)

In [18]:
sy.playing()

True

In [19]:
sy.set(kfreq=850)

Synth Synth(■ baz=502.0002 start=35.742 dur=2.000 p5=800 kfreq=0.1) has already stopped, cannot set param 'kfreq'


In [20]:
sy = s.sched('baz', dur=3, kfreq=1000, iamp=0.01)
sy.automate('kfreq', (0, 0, 2, 200, 3, 800), overtake=True)

11

This should not work

In [21]:
test = s.defInstr('test', r'''
|kfreq=1000|
pset 0, 0, 0, 0, 0.1, 0.5
iamp = p5
outch 1, oscili:a(iamp, kfreq * p6)
''')
test

Instr(test, kfreq=1000)

In [22]:
test = s.defInstr('output', r'''
|ichan=1|
ain = inch:a(ichan)
aenv = follow2(ain, 0.1, 0.2)
kenv = downsamp(aenv, 16)
if changed(kenv) == 1 then
    println "kenv: %f dB", dbamp(kenv)
endif
''')
test

Instr(output, ichan=1)

In [23]:
s.defInstr('sine', r'''
        |iamp=0.1, kfreq=1000|
        a0 = oscili:a(kamp, kfreq)
        outch 1, a0
    ''')

Instr(sine, iamp=0.1, kfreq=1000)

In [25]:
r = s.makeRenderer()
r.sched('sine', dur=1, iamp=0.01, kfreq=440)
r.writeCsd("~/tmp/sine.csd")

In [26]:
with s.rendering('~/tmp/test.wav'):
    s.sched('sine', dur=1, iamp=0.01, kfreq=440)

In [27]:
instr = s.defInstr('pfields', r'''
kfreq = p5
outch 1, oscili:a(0.1, kfreq)
''')
instr

Instr(pfields, kfreq:5)

In [28]:
instr.parsedCode

ParsedInstrBody(pfieldIndexToName={5: 'kfreq'}, pfieldLines=['kfreq = p5'], body='outch 1, oscili:a(0.1, kfreq)', lines=['', 'kfreq = p5', 'outch 1, oscili:a(0.1, kfreq)'], pfieldIndexToValue={}, pfieldsUsed={5}, outChannels=set())

In [29]:
instr.parsedCode.pfieldNameToValue

{'kfreq': 0.0}

In [30]:
instr.parsedCode.pfieldIndexToValue

{}

In [31]:
instr.dynamicPfields()

{'kfreq': 0.0}

In [32]:
instr.aliases

{}